<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/ejemyr/Lab-2/ejemyr_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Christoffer Ejemyr**

# **Abstract**

This lab aims to implement efficient algorithms to both multiply and factorize matrices. The focus lies both in mathematical accuracy and computational cost.

All methods were implemented in a satisfactory way and the accuracy was generally high.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [2]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Christoffer Ejemyr (ejemyr@kth.se)
# In collaboration with Leo Enge (leoe@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import unittest
import numpy as np
import scipy.sparse as sparse

# **Introduction**

This lab aims to implement efficient algorithms to both multiply and factorize matrices. The focus lies both in mathematical accuracy and computational cost.


# **Methods**

## Sparce matrix class
This class saves and handles sparce matrices in CRS format. I've overvritten the numpy method *dot(self, other)* to manually define the matrix-vector product. The algorithm uses the liniarity of matrix-vector multiplication by adding the contribution from each non-zero element in the sparse matrix.

In [0]:
class SparseMatrix(sparse.csr_matrix):
    def dot(self, other):
        if not(type(other) == np.ndarray and other.ndim == 1):
            raise Exception("Vector format not recognized.")
        
        if other.size != self.shape[1]:
            raise Exception("Vector is of wrong length.")

        b = np.zeros(self.shape[0])
        for i in range(self.shape[0]):
            for j in range(self.indptr[i], self.indptr[i + 1]):
                b[i] += self.data[j] * other[self.indices[j]]
        return b

    def __str__(self):
        return str(self.todense())

## QR-factorization
The QR-factorization factorizes a matrix $A$ into a matrix $Q$ with normalized column vectors spanning $\text{Range}(A)$ and an upper triangonal square matrix $R$ with values scaling the column vectors of $Q$ back to $A$.

The algorithm used is the ordenary Gram-Schmidt method. Its simlisity makes it easely adaptable and is here implemented for all $m \times n$ matrices where $m \geq n$.



In [0]:
def QR_factorization(A):
    if not(type(A) == np.ndarray and A.ndim == 2 and A.shape[1] <= A.shape[0]):
        raise Exception("Matrix format not recognized.")

    R = np.zeros((A.shape[1], A.shape[1]))
    Q = np.zeros(A.shape)
    v = np.zeros(A.shape[0])
    v[:] = A[:, 0]

    for i in range(A.shape[1]):
        R[i, i] = np.linalg.norm(v)
        Q[:, i] = v / R[i, i]
        for j in range(i + 1, A.shape[1]):
            R[i, j] = Q[:, i].dot(A[:, j])

        if i + 1 != A.shape[1]:
            v[:] = A[:, i + 1]
            for j in range(i + 1):
                v[:] -= R[j, i + 1] * Q[:, j]
    
    return Q, R

## Matrix equation solvers
Below three functions for solving or optimizing matrix equations are implemented. All functions solve equations on the form
$$ A x = b. $$
The *backwards_substitution* function works for square, upper triangular matrices $A$. It then uses posibility to explicitly calculate each component of $x$ by backtracking from the equation of the last row in $Ax=b$.

The *eq_sys_solver* function only takes square non-singular matrices. Using QR-factorization and the fact that for square matrices $Q^{-1} = Q^T$ you get

\begin{align}
Ax&=b \\
QRx&=b \\
Rx&=Q^Tb
\end{align}

Since $R$ is upper triangonal the system can be solved by backwards substitution.

In a similar manner the *least_squares* uses the fact that the solution $x$ to $A^TAx = A^Tb$ will minimise $||Ax - b||$ and thus be the least square solution. Since $A^TA$ is a square matrix we can use the same method as in the *eq_sys_solver* function to solve this new matrix equation.

In [0]:
def backwards_substitution(U, b):
    if not(type(U) == np.ndarray and U.ndim == 2):
        raise Exception("Matrix format not recognized.")
    if (U != np.triu(U)).any():
        raise Exception("Matrix is not upper triangular.")
    if np.linalg.det(U) == 0:
        raise Exception("Matrix is singular")
    if not(type(b) == np.ndarray and b.ndim == 1):
        raise Exception("Vector format not recognized.")
    if len(b) != U.shape[1]:
        raise Exception("Vector and matrix formats does not match.")
    
    n = U.shape[0]
    x = np.zeros(n)
    x[-1] = b[-1] / U[-1, -1]

    for i in range(n - 2, -1, -1):
        s = 0
        for j in range(i + 1, n):
            s += U[i, j] * x[j]
        x[i] = (b[i] - s) / U[i, i]

    return x

def eq_sys_solver(A, b):
    if A.shape[0] != A.shape[1]:
        raise Exception("Matrix not square.")
    if np.linalg.det(A) == 0:
        raise Exception("Matrix is singular.")
    if not(type(A) == np.ndarray and A.ndim == 2):
        raise Exception("Matrix format not recognized.")
    if not(type(b) == np.ndarray and b.ndim == 1):
        raise Exception("Vector format not recognized.")
    if len(b) != A.shape[0]:
        raise Exception("Vector and matrix formats does not match.")
    
    Q, R = QR_factorization(A)
    return backwards_substitution(R, Q.transpose().dot(b))

def least_squares(A, b):
    if not(type(A) == np.ndarray and A.ndim == 2):
        raise Exception("Matrix format not recognized.")
    if not(type(b) == np.ndarray and b.ndim == 1):
        raise Exception("Vector format not recognized.")
    if len(b) != A.shape[0]:
        raise Exception("Vector and matrix formats does not match.")
    
    Q, R = QR_factorization(A.transpose().dot(A))
    return backwards_substitution(R, Q.transpose().dot(A.transpose().dot(b)))

## QR eigenvalue algorithm
This eigenvalue algorithm will, after many itterations, converge A and U to the Schur factorization matrices. Since only square matrices will have eigenvalues we limit $A_{in}$ to beeing a square matrix. The method returns the approximation of the eigenvalues and the corresponding eigenvectors.

In [0]:
def eigen_vals_vecs(A_in, ittr :int):
    A = A_in.copy()
    if not(type(A) == np.ndarray and A.ndim == 2):
        raise Exception("Matrix format not recognized.")
    if A.shape[0] != A.shape[1]:
        raise Exception("Matrix not square.")
    U = np.eye(A.shape[0])
    for i in range(ittr):
        Q, R = QR_factorization(A)
        A = R.dot(Q)
        U = U.dot(Q)
    return A.diagonal(), U

## Block matrix matrix multiplication
My implementation divides the matrices into blocks using the folowing algorithm.

Given a dimention of length $N$ that is to be divided into $n$ blocks the fisrt block will be of size
$$d_1 = \text{ceil}(N / n).$$
The next block will then be of size
$$d_2 = \text{ceil}(\frac{N - d_1}{n - 1}).$$
Continuing with the $i$:th block beeing of size
$$d_i = \text{ceil}(\frac{N - d_1 - d_2 - \ldots - d_{i-1}}{n - (i - 1)}.)$$

This method will garantuee that the blocks will be of sizes differing by maximally one element and that the larges blocks will be first follwed by the smaller blocks.

I chose this method since it is very deterministic and not dependent on coinsidences between matrix sizes and bock numbers.

In [0]:
def blocked_matrix_matrix(A, B, m :int, n :int, p :int):
    if not(type(A) == np.ndarray and A.ndim == 2 and type(B) == np.ndarray and B.ndim == 2):
        raise Exception("Matrix format not recognized.")
    if A.shape[1] != B.shape[0]:
        raise Exception("Matrix format do not argree.")
    if m > A.shape[0] or m < 1 or n > B.shape[1] or n < 1 or p > A.shape[1] or p < 1:
        raise Exception("Invlid number of blocks.")

    C = np.zeros((A.shape[0], B.shape[1]))

    idx_i, idx_j, idx_k = 0, 0, 0
    step_i, step_j, step_k = 0, 0, 0

    for i in range(m):
        idx_i += step_i
        step_i = int(np.ceil((A.shape[0] - idx_i) / (m - i)))
        idx_j = 0
        step_j = 0
        for j in range(n):
            idx_j += step_j
            step_j = int(np.ceil((B.shape[1] - idx_j) / (n - j)))
            idx_k = 0
            step_k = 0
            for k in range(p):
                idx_k += step_k
                step_k = int(np.ceil((A.shape[1] - idx_k) / (p - k)))
                C[idx_i : idx_i + step_i, idx_j : idx_j + step_j] += A[idx_i : idx_i + step_i, idx_k : idx_k + step_k].dot(B[idx_k : idx_k + step_k, idx_j : idx_j + step_j])
                
    return C

# Tests
Testing the algorithms mainly consists of two parts: checking raises and other assertions and testing for accuracy and floating point precition.

Generally the first is done for some common mistakes and checks that exceptions are raised. The second test is done by multiple times generating random input data and testing either against nown results, like norm equal to zero, or against other algorithms that are known to be accurate.

Most of the accurasy testing methods are strait forward and easy to understand. One that is more interesting is the method to test the least squares solution. Since the norm will not always be zero (only in special cases) we must instead check that the norm of the error is the smallest one for all x. What I did was to repeatidly add a small vector $v$ to the solution $x$ and check that the norm of the error was never smaller than the least squares solution.

In [16]:
class TestSparseMatrix(unittest.TestCase):

    def test_exceptions(self):
        A = SparseMatrix([[1,0,0],[0,1,0],[0,0,1]])
        B = SparseMatrix([[0,0,0],[0,0,0],[0,0,1]])
        v = np.array([1, 4])
        u = [1, 3, 4]
        with self.assertRaises(Exception):
            A.dot(B)
        with self.assertRaises(Exception):
            A.dot(v1)
        with self.assertRaises(Exception):
            A.dot(v2)

    def test_accuracy(self):
        max_n = 10
        for i in range(num_of_tests):
            n = np.random.randint(1, max_n)
            M = np.random.rand(np.random.randint(1, max_n), n)
            M_spase = SparseMatrix(M)
            v = np.random.rand(n)
            for i in range(7):
                np.testing.assert_almost_equal(M.dot(v), M_spase.dot(v), decimal=i)


class TestQRFactorization(unittest.TestCase):
    def test_exceptions(self):
        with self.assertRaises(Exception):
            QR_factorization(np.array([1]))
        with self.assertRaises(Exception):
            QR_factorization([[1, 2], [3, 4]])
            
    def test_accuracy(self):
        max_n = 10
        for i in range(num_of_tests):
            n = np.random.randint(1, max_n)
            M = np.random.rand(n, n)

            Q, R = QR_factorization(M)
            M_reconstructed = Q.dot(R)

            for i in range(7):
                np.testing.assert_almost_equal(
                    np.linalg.norm(Q.transpose().dot(Q)-np.eye(Q.shape[0]), 'fro'),
                    0,
                    decimal=i)
                np.testing.assert_almost_equal(
                    np.linalg.norm(Q.dot(R) - M, 'fro'),
                    0,
                    decimal=i)
                np.testing.assert_almost_equal(R, np.triu(R), decimal=i)


class TestBackwardsSub(unittest.TestCase):
    def test_exceptions(self):
        with self.assertRaises(Exception):
            backwards_substitution(np.array([[1, 2, 3], [1, 2, 3], [0, 0, 1]]),
                                  np.array([1, 2, 3]))
            
    def test_accuracy(self):
        max_n = 10
        for i in range(num_of_tests):
            n = np.random.randint(1, max_n)
            U = np.triu(np.random.rand(n, n))
            x = np.random.rand(n)
            b = U.dot(x)
            for i in range(7):
                np.testing.assert_almost_equal(x, backwards_substitution(U, b), decimal=i)


class TestEqSolver(unittest.TestCase):
    def test_accuracy(self):
        max_n = 10
        for i in range(num_of_tests):
            n = np.random.randint(1, max_n)
            A = np.random.rand(n, n)
            x_true = np.random.rand(n)
            b = A.dot(x_true)
            x = eq_sys_solver(A, b)

            for i in range(7):
                np.testing.assert_almost_equal(
                    np.linalg.norm(x - x_true),
                    0,
                    decimal=i)
                np.testing.assert_almost_equal(
                    np.linalg.norm(A.dot(x) - b),
                    0,
                    decimal=i)


class TestLeastSquare(unittest.TestCase):
    def test_accuracy(self):
        max_dim = 10
        for i in range(num_of_tests):
            A = np.zeros((1,1))
            while np.linalg.det(A.transpose().dot(A)) == 0:
                m = np.random.randint(1, max_dim)
                n = np.random.randint(1, m + 1)
                A = np.random.rand(m, n)
                b = np.random.rand(m)
            x = least_squares(A, b)

            for i in range(100):
                diff_vec = 0.01 * (2 * np.random.rand(n) - 1)
                self.assertTrue(np.linalg.norm(A.dot(x) - b) <= np.linalg.norm(A.dot(x + diff_vec) - b))


class TestEigenValues(unittest.TestCase):
    def test_accuracy(self):
        max_n = 10
        for i in range(num_of_tests):
            n = np.random.randint(1, max_n)
            A = np.random.rand(n, n)
            A = A.transpose().dot(A)
            eigen_vals, eigen_vectors = eigen_vals_vecs(A, 100)

            for i in range(4):
                for i, e in enumerate(eigen_vals):
                    np.testing.assert_almost_equal(np.linalg.det(A - e * np.eye(A.shape[0])), 0, decimal=i)
                    np.testing.assert_almost_equal(np.linalg.norm(A.dot(eigen_vectors[:, i]) - e * eigen_vectors[:, i]), 0, decimal=i)


class TestBlockedMatrixMult(unittest.TestCase):
    def test_accuracy(self):
        max_dim = 50
        for i in range(num_of_tests):
            M = np.random.randint(1, max_dim + 1)
            N = np.random.randint(1, max_dim + 1)
            P = np.random.randint(1, max_dim + 1)
            A = np.random.rand(M, P)
            B = np.random.rand(P, N)

            m = np.random.randint(1, M + 1)
            n = np.random.randint(1, N + 1)
            p = np.random.randint(1, P + 1)

            for i in range(7):
                np.testing.assert_almost_equal(
                    blocked_matrix_matrix(A, B, m, n, p),
                    A.dot(B),
                    decimal=i
                    )


num_of_tests = 100

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 15.323s

OK


# **Results**

All test passed with an accuracy of up to seven decimals, except the eigenvalue finder that only had an accuracy of someware around four decimal places. The accuracy of that method did increase with the number of itterations, but is very dependent on the matrix given.

# **Discussion**

No suprices were encounterd and all methods implemented in a satisfactory way. I'm especially pleased with the block-size algorithm in the blocked matrix-matrix multiplication algorithm that I figured out on my own.